# Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
| **`Tracking server`**   | yes, remote server (EC2) |
| :---------------------: | :--------------: | 
| **`Backend store`**     | postgresql database  |
| **`Artifact store`**    | s3 bucket |

The experiment can be explored by accessing the remote server.

The example uses AES to host a remote server. In order to run the example you'll need an AWS account. Foow the steps in the file [`mlflow_on_aws.md`](https://github.com/joweyel/mlops-zoomcamp/blob/main/02-experiment-tracking/mlflow_on_aws.md) to create a new AWS account and launch the tracking server.

In [ ]:
import os
import mlflow

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/

TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

## TODO (on AWS)